In [1]:
from zipfile import ZipFile
file_name = "datasets.zip"

with ZipFile(file_name, 'r') as zip1:
  zip1.extractall()
  print('Done')

Done


In [2]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

!ls

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,693 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,486 kB]
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [884 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:13 https://cloud.r-project.org

In [3]:
import pandas as pd
import os


In [4]:
reviews_file = 'Datasets/reviews_32618_for_1098_users_with_location.xlsx'
user_file = 'Datasets/users_full_7034.xlsx'
personality_file = 'Datasets/pers_scores_1098.xlsx'

u_df = pd.read_excel(user_file)
r_df = pd.read_excel(reviews_file)
p_df = pd.read_excel(personality_file)

In [5]:
import numpy as np
from collections import Counter

In [6]:
user_df=u_df.copy(True)
null_age = user_df.ageRange.isnull()
null_gender = user_df.gender.isnull()
null_style = user_df.travelStyle.isnull()
one_hund_pt = (user_df.totalPoints > 100)

In [7]:
user_df.head(5)

,username,ageRange,gender,location,travelStyle,reviewerBadge,registerDate,numHotelsReviews,numRestReviews,numAttractReviews,numFirstToReview,numRatings,numPhotos,numForumPosts,numArticles,numCitiesBeen,totalPoints,contribLevel,numHelpfulVotes
0,007solotraveler,35-49,male,liverpool,NaN,Contributor,2010-06-01,10.0,1.0,1.0,NaN,6.0,NaN,NaN,NaN,8.0,1240.0,3,10
1,0BKI0,25-34,female,Bristol,"Foodie, Nature Lover, Urban Explorer",Contributor,2009-08-01,11.0,7.0,NaN,NaN,1.0,10.0,NaN,NaN,11.0,2117.0,3,12
2,1010101,NaN,NaN,uk,NaN,NaN,2009-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0
3,10179k79,NaN,NaN,"Berlin, Germany",NaN,Top Contributor,2008-05-01,54.0,1.0,NaN,3.0,2.0,8.0,NaN,NaN,48.0,5808.0,5,58
4,10onwoodford,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [8]:
user_df.shape

(7034, 19)

In [9]:
null_age.value_counts()

False    4786
True     2248
Name: ageRange, dtype: int64

In [10]:
null_gender.value_counts()

False    5299
True     1735
Name: gender, dtype: int64

In [11]:
null_style.value_counts()

True     3532
False    3502
Name: travelStyle, dtype: int64

In [12]:
one_hund_pt.value_counts()

True     6949
False      85
Name: totalPoints, dtype: int64

In [13]:
user_filtered = user_df[one_hund_pt][~null_age][~null_gender][~null_style]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [14]:
user_filtered.shape

(2611, 19)

In [15]:
user_filtered = user_filtered[['username', 'ageRange', 'gender', 'travelStyle']]

In [16]:
user_filtered.head(5)

,username,ageRange,gender,travelStyle
1,0BKI0,25-34,female,"Foodie, Nature Lover, Urban Explorer"
7,14beacon,35-49,male,"Foodie, Trendsetter, Like a Local, Luxury Trav..."
12,1975mark,25-34,male,"Foodie, Beach Goer, Thrifty Traveller, Nightli..."
15,19Cam,25-34,male,"Vegetarian, Urban Explorer, Art and Architectu..."
17,1Angie,18-24,female,"Beach Goer, Vegetarian, Urban Explorer"


In [17]:
reviews_df=r_df.copy(True)
reviews_df.head(5)


,id,username,type,date,title,text,rating,helpfulness,total_points,taObject,taObjectUrl,taObjectCity
0,1,007solotraveler,Hotels,2013-04-08,Good Choice and would be suitable for a family,Decent Hotel next to station so good location ...,4,1.0,101,"Radisson Blu Royal Viking Hotel, Stockholm",http://www.tripadvisor.com/Hotel_Review-g18985...,Stockholm
1,2,007solotraveler,Hotels,2013-04-08,Modern and Chic - suitable both family and bus...,Excellent Hotel - well situated for getting ro...,5,0.0,100,Radisson Blu Waterfront Hotel,http://www.tripadvisor.com/Hotel_Review-g18985...,Stockholm
2,3,007solotraveler,Attractions,2013-02-20,Brilliant - Well worth a visit(s),Great Museum - abslutely worth making the time...,5,1.0,101,Vasa Museum,http://www.tripadvisor.com/Attraction_Review-g...,Stockholm
3,4,007solotraveler,Hotels,2012-07-09,Family stay,Stayed for 3 nights in MIami - stayed here due...,4,0.0,100,Hilton Miami Airport,http://www.tripadvisor.com/Hotel_Review-g34438...,Miami
4,5,007solotraveler,Restaurants,2011-09-07,Great Prime Rib!,I always visit Hunters when in San Diego. The ...,5,1.0,101,Hunter Steakhouse,http://www.tripadvisor.com/Restaurant_Review-g...,San Diego


In [18]:
reviews_df.shape

(32580, 12)

In [19]:
review_filtered = reviews_df[['id', 'username', 'type', 'title', 'text', 'rating', 'taObjectCity']]

In [20]:
review_filtered.head(5)

,id,username,type,title,text,rating,taObjectCity
0,1,007solotraveler,Hotels,Good Choice and would be suitable for a family,Decent Hotel next to station so good location ...,4,Stockholm
1,2,007solotraveler,Hotels,Modern and Chic - suitable both family and bus...,Excellent Hotel - well situated for getting ro...,5,Stockholm
2,3,007solotraveler,Attractions,Brilliant - Well worth a visit(s),Great Museum - abslutely worth making the time...,5,Stockholm
3,4,007solotraveler,Hotels,Family stay,Stayed for 3 nights in MIami - stayed here due...,4,Miami
4,5,007solotraveler,Restaurants,Great Prime Rib!,I always visit Hunters when in San Diego. The ...,5,San Diego


In [21]:
review_filtered.shape

(32580, 7)

In [22]:
merged_df = pd.merge(review_filtered, user_filtered, on=['username'])
merged_df.head(5)

,id,username,type,title,text,rating,taObjectCity,ageRange,gender,travelStyle
0,13,0BKI0,Restaurants,Unhygienic & terrible service,I have visited this restaurant twice before an...,1,Bristol,25-34,female,"Foodie, Nature Lover, Urban Explorer"
1,14,0BKI0,Restaurants,Beautiful resturant overlooking the beach and ...,If you are in Alvor you must visit Canico! Bea...,5,Alvor,25-34,female,"Foodie, Nature Lover, Urban Explorer"
2,15,0BKI0,Restaurants,Fantastic food & lovely staff,The food here was fantastic - we visited twice...,5,Alvor,25-34,female,"Foodie, Nature Lover, Urban Explorer"
3,16,0BKI0,Restaurants,Food was ok...expected better!,"The food here is good, however nothing special...",3,Bristol,25-34,female,"Foodie, Nature Lover, Urban Explorer"
4,17,0BKI0,Restaurants,Amazing restaurant for sushi lovers!,Loved this restaurant and can&acute;t wait to ...,5,Bristol,25-34,female,"Foodie, Nature Lover, Urban Explorer"


In [23]:
merged_df.shape

(15625, 10)

In [24]:
popular_city = []
for item, value in Counter(merged_df.taObjectCity).items():
    if value >= 10:
        popular_city.append(item)

In [25]:
len(popular_city)

277

In [26]:
merged_df = merged_df[merged_df.taObjectCity.isin(popular_city)]

In [27]:
merged_df.shape

(8425, 10)

In [28]:
personality_df=p_df.copy(True)
personality_df.head(5)

,username,open,cons,extra,agree,neuro
0,007solotraveler,0.63272,0.55666,0.56360,0.58108,0.45881
1,0BKI0,0.64006,0.55189,0.56220,0.57943,0.46188
2,10179k79,0.66435,0.56099,0.56880,0.58187,0.45011
3,124_10,0.66003,0.57005,0.57221,0.59019,0.44860
4,12ReasonsWhy,0.65036,0.55429,0.56510,0.58271,0.45919


In [29]:
personality_df.shape

(1098, 6)

In [30]:
final_df=pd.merge(merged_df, personality_df, on=['username'])
final_df.head(5)

,id,username,type,title,text,rating,taObjectCity,ageRange,gender,travelStyle,open,cons,extra,agree,neuro
0,13,0BKI0,Restaurants,Unhygienic & terrible service,I have visited this restaurant twice before an...,1,Bristol,25-34,female,"Foodie, Nature Lover, Urban Explorer",0.64006,0.55189,0.5622,0.57943,0.46188
1,16,0BKI0,Restaurants,Food was ok...expected better!,"The food here is good, however nothing special...",3,Bristol,25-34,female,"Foodie, Nature Lover, Urban Explorer",0.64006,0.55189,0.5622,0.57943,0.46188
2,17,0BKI0,Restaurants,Amazing restaurant for sushi lovers!,Loved this restaurant and can&acute;t wait to ...,5,Bristol,25-34,female,"Foodie, Nature Lover, Urban Explorer",0.64006,0.55189,0.5622,0.57943,0.46188
3,18,0BKI0,Restaurants,Great food & great value!,Loved this little place! Was staying next door...,5,London,25-34,female,"Foodie, Nature Lover, Urban Explorer",0.64006,0.55189,0.5622,0.57943,0.46188
4,19,0BKI0,Hotels,Lovely hotel & great location,This is a lovely hotel in a great location - w...,4,London,25-34,female,"Foodie, Nature Lover, Urban Explorer",0.64006,0.55189,0.5622,0.57943,0.46188


In [31]:
final_df.shape

(8425, 15)

In [39]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Recommendation_system').getOrCreate()

In [40]:
spark_df = spark.read.

TypeError: ignored

In [35]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [36]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

In [37]:
indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in ['username', 'type', 'title', 'text', 'taObjectCity', 'ageRange', 'gender', 'travelStyle'] ]
pipeline = Pipeline(stages=indexer)
transformed = pipeline.fit(final_df).transform(final_df)
transformed.show()

AttributeError: ignored